In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

from tqdm.notebook import tqdm
tqdm.pandas()
from geoid.censusnames import stusab
import rowgenerators as rg
from geoid.acs import Puma

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg

# Tract Income Distributions
`civicknowledge.com-income_dist-tract-1.1.1` Last Update: 2021-02-19T04:02:37

_Estimated portions of households per tracts in income ranges_




 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [tract_income_dist](notebooks/Extract.ipynb)**. Estimated portions of households per tracts in income ranges

## References
<ul><li> <strong>pums</string>, <em>pums:{st}/h/2019/5</em>. PUMS template</li><li> <strong>puma_tract_map</string>, <em>data/puma_tract_map.csv</em>. Puma to tract map</li><li> <strong>us_tracts</string>, <em>metapack+http://library.metatab.org/civicknowledge.com-geohash-us.csv#us_tracts</em>. US Tracts, with tract numbers</li><li> <strong>households</string>, <em>census://2019/5/US/tract/B11001</em>. US Households by tract</li><li> <strong>median_income</string>, <em>census://2019/5/US/tract/B19013</em>. US Household median income by tract</li><ul>

In [3]:
ptm = pkg.reference('puma_tract_map').dataframe()
ptm = ptm.rename(columns={'tract':'geoid','puma':'PUMA'})

In [4]:
%%time
from pathlib import Path
p = Path('pums.pkl')
if p.exists():
    pums = pd.read_pickle(p)
else:
    frames = [rg.dataframe(pkg.reference('pums').url.format(st=st), low_memory=False) for st in tqdm(list(stusab.values()))]
    pums = pd.concat(frames)
    pums = pums[['ST','PUMA','HINCP', 'WGTP']]
    pums['PUMA'] = pums.progress_apply(lambda r: str(Puma(r.ST, r.PUMA)), axis=1)
    pums.to_pickle('pums.pkl')

CPU times: user 707 ms, sys: 294 ms, total: 1 s
Wall time: 1 s


In [5]:
pums.head()

,ST,PUMA,HINCP,WGTP
0,1,79500US0101000,NaN,14
1,1,79500US0102701,52450.0,9
2,1,79500US0100400,NaN,11
3,1,79500US0101000,8800.0,15
4,1,79500US0101000,13200.0,29


In [6]:
p = Path('hh.csv')
if not p.exists():
    hh = pkg.reference('households').dataframe().rename(columns={'b11001_001':'households'})
    hh = hh[['households']].copy()
    hh.to_csv(str(p))
else:
    hh = pd.read_csv(p, index_col=False)

p = Path('mi.csv')
if not p.exists():
    mi = pkg.reference('median_income').dataframe().rename(columns={'b19013_001':'median_income'})
    mi = mi[['median_income']].copy()
    mi.to_csv(p)
else:
    mi = pd.read_csv(p, index_col=False)


In [7]:
# This may not be the correct weighting -- maybe
# The weights are only vild within the PUMA?
samp = pums.dropna()
samp = samp.sample(int(10e6), replace=True, weights=samp.WGTP)

In [8]:
step = 5_000

# Clip so we don't deal with crazy extremes
samp['HINCP'] = samp.HINCP.clip(-step, 500_000)

# Quantize the median incomes of each puma. This becomes the index we will use
# to match tracts to PUMA distributions
samp['medinc'] = samp.groupby('PUMA').HINCP.transform(lambda g: (g.median()/step).round()*step).astype(int)
samp.head()

,ST,PUMA,HINCP,WGTP,medinc
280612,42,79500US4201200,79000.0,23,55000
80129,42,79500US4203004,43700.0,29,75000
50527,21,79500US2102300,55300.0,19,55000
139766,39,79500US3904106,20800.0,21,35000
417992,12,79500US1209903,113000.0,14,45000


In [9]:
# Build the list of bin boundaries
inc_bins = np.arange(-step, samp.HINCP.max()+step, step)

# Assign the household incomes to bins
samp['inc_bin'] = pd.cut(samp.HINCP, inc_bins).apply(lambda e: e.left)
samp.head()

,ST,PUMA,HINCP,WGTP,medinc,inc_bin
280612,42,79500US4201200,79000.0,23,55000,75000.0
80129,42,79500US4203004,43700.0,29,75000,40000.0
50527,21,79500US2102300,55300.0,19,55000,55000.0
139766,39,79500US3904106,20800.0,21,35000,20000.0
417992,12,79500US1209903,113000.0,14,45000,110000.0


In [10]:
# Group by the median income index ( and across pums ) and count up the number of people

medinc_bins = samp.groupby('medinc').inc_bin.value_counts().unstack().fillna(0)
medinc_bins = medinc_bins.divide(samp.groupby('medinc').inc_bin.count(), axis=0)
medinc_bins.columns = list(medinc_bins.columns)
medinc_bins.sort_index(level=['medinc', ]).head(10)

,-5000.0,0.0,5000.0,10000.0,15000.0,20000.0,25000.0,30000.0,35000.0,40000.0,...,450000.0,455000.0,460000.0,465000.0,470000.0,475000.0,480000.0,485000.0,490000.0,495000.0
medinc,,,,,,,,,,,,,,,,,,,,,
15000,0.088671,0.103908,0.152933,0.136103,0.104114,0.081528,0.064698,0.049950,0.042807,0.031655,...,0.000000,0.000000,0.000000,0.000000,0.000128,0.000026,0.000000,0.000000,0.000000,0.000103
20000,0.069558,0.092035,0.127052,0.116893,0.105368,0.086542,0.063145,0.056891,0.050383,0.036985,...,0.000063,0.000000,0.000000,0.000095,0.000000,0.000000,0.000000,0.000127,0.000000,0.000254
25000,0.043350,0.059833,0.116741,0.111458,0.089257,0.076518,0.063996,0.057526,0.055921,0.043049,...,0.000017,0.000050,0.000033,0.000050,0.000084,0.000033,0.000000,0.000000,0.000000,0.000769
30000,0.037201,0.043019,0.096689,0.093902,0.085833,0.072641,0.069364,0.055294,0.053493,0.042523,...,0.000177,0.000051,0.000097,0.000199,0.000114,0.000165,0.000097,0.000057,0.000097,0.001601
35000,0.030288,0.035966,0.076285,0.082407,0.077197,0.071259,0.067089,0.057766,0.055193,0.045704,...,0.000068,0.000116,0.000083,0.000118,0.000151,0.000124,0.000116,0.000080,0.000083,0.001839
40000,0.023141,0.028096,0.057141,0.073719,0.071088,0.068494,0.065478,0.058251,0.058560,0.048049,...,0.000190,0.000194,0.000140,0.000139,0.000179,0.000186,0.000095,0.000141,0.000153,0.002292
45000,0.017643,0.025193,0.046662,0.063511,0.064057,0.063180,0.061475,0.055276,0.056670,0.048957,...,0.000200,0.000156,0.000232,0.000216,0.000146,0.000212,0.000210,0.000183,0.000157,0.003079
50000,0.014595,0.022092,0.039353,0.054709,0.056191,0.056802,0.056719,0.052381,0.054605,0.047072,...,0.000236,0.000208,0.000226,0.000250,0.000219,0.000214,0.000205,0.000178,0.000178,0.003635
55000,0.013042,0.019524,0.033262,0.046953,0.050596,0.051754,0.052475,0.047975,0.051674,0.045267,...,0.000318,0.000262,0.000334,0.000279,0.000253,0.000320,0.000321,0.000221,0.000218,0.004607


In [11]:

mi_max = max(medinc_bins.index)
mi_min = min(medinc_bins.index)

In [34]:
mi['medinc'] = (mi.median_income/step).clip(mi_min, mi_max).round().fillna(0).astype(int)

t = mi.merge(hh).merge(medinc_bins.reset_index())

t.loc[:,-5000:]= t.loc[:,-5000:].multiply(t.households, axis=0)
t.head()

,geoid,median_income,medinc,households,-5000.0,0.0,5000.0,10000.0,15000.0,20000.0,...,450000.0,455000.0,460000.0,465000.0,470000.0,475000.0,480000.0,485000.0,490000.0,495000.0
0,14000US01001020100,60208.0,15000,709,62.867982,73.670855,108.429507,96.497161,73.816593,57.803566,...,0.0,0.0,0.0,0.0,0.091087,0.018217,0.0,0.0,0.0,0.072869
1,14000US01001020200,43958.0,15000,688,61.005884,71.488784,105.217914,93.638994,71.630206,56.091472,...,0.0,0.0,0.0,0.0,0.088389,0.017678,0.0,0.0,0.0,0.070711
2,14000US01001020300,55345.0,15000,1360,120.593027,141.315039,207.988900,185.100337,141.594594,110.878491,...,0.0,0.0,0.0,0.0,0.174722,0.034944,0.0,0.0,0.0,0.139777
3,14000US01001020400,59663.0,15000,1675,148.524500,174.046096,256.162800,227.972841,174.390401,136.559906,...,0.0,0.0,0.0,0.0,0.215191,0.043038,0.0,0.0,0.0,0.172152
4,14000US01001020500,66108.0,15000,4483,397.513631,465.820088,685.598705,610.150595,466.741592,365.491380,...,0.0,0.0,0.0,0.0,0.575940,0.115188,0.0,0.0,0.0,0.460752


In [25]:
tracts = pkg.reference('us_tracts').dataframe()
tracts = tracts[['geoid','tract_id']]
tract_income_dist = tracts.merge(t)#.drop(columns=['medinc','households'])
tract_income_dist.head()

In [44]:
income_ranges = tract_income_dist[['tract_id']].copy()
income_ranges['lt25k'] = tract_income_dist.loc[:,-5_000:20_000].sum(axis=1)
income_ranges['25k_50k'] = tract_income_dist.loc[:,25_000:45_000].sum(axis=1)
income_ranges['50k_75k'] = tract_income_dist.loc[:,50_000:70_000].sum(axis=1)
income_ranges['75k_120k'] = tract_income_dist.loc[:,75_000:115_000].sum(axis=1)
income_ranges['gt120k'] = tract_income_dist.loc[:,120_000:].sum(axis=1)
income_ranges['gt50k'] = tract_income_dist.loc[:,50_000:].sum(axis=1)
income_ranges['gt60k'] = tract_income_dist.loc[:,60_000:].sum(axis=1)
income_quartiles = income_ranges
income_quartiles.describe()


,tract_id,lt25k,25k_50k,50k_75k,75k_120k,gt120k,gt50k,gt60k
count,72913.000000,72913.000000,72913.000000,72913.000000,72913.000000,72913.000000,72913.000000,72913.000000
mean,36968.733175,1115.869975,365.239123,112.966312,53.281943,24.965168,191.213423,129.337619
std,21361.834657,544.485143,178.217248,55.121546,25.998751,12.181673,93.301971,63.109873
min,0.000000,8.007092,2.620828,0.810607,0.382333,0.179141,1.372080,0.928081
25%,18476.000000,747.995812,244.829004,75.724171,35.716231,16.734782,128.175184,86.698271
50%,36986.000000,1038.252884,339.834014,105.108662,49.575786,23.228654,177.913102,120.341222
75%,55472.000000,1387.228629,454.058429,140.437601,66.239112,31.036229,237.712942,160.790103
max,74000.000000,14151.867237,4632.094864,1432.679694,675.740898,316.617308,2425.037899,1640.306534
